In [0]:
!git clone https://github.com/marcingrzyb/computational-intelligence.git

Cloning into 'computational-intelligence'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 13 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), done.


In [0]:
with open('/content/computational-intelligence/project1/diagnosis.data', encoding = 'utf-16') as fin, open('/content/computational-intelligence/project1/diagnosis2.data','w+') as fout:
    fout.write('a1;a2;a3;a4;a5;a6;d1;d2\n')
    for line in fin:
        line=line.replace(',','.')
        fout.write(line.replace('\t', ';'))



In [0]:
import pandas as pd
df1=pd.read_csv('/content/computational-intelligence/project1/diagnosis2.data',sep=';')

df1=df1.replace('no',False)
df=df1.replace('yes',True)

d1=df.pop('d1') #decision1
d2=df.pop('d2')#decision2

In [0]:
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df, d1, test_size=0.3, random_state=41, shuffle=True)#d1 value prediction

In [0]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

clf = SVC()
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)
cm=confusion_matrix(y_test, y_pred)
print(cm)

tp = float(cm[0][0])/np.sum(cm[0])

print(tp)

[[17  0]
 [19  0]]
1.0
